# TextRNN
RNN的基本公式如下：
$$h_t = \tanh(W_{ih}x_x + W_{hh}h_{t-1} + b_h)$$
只需按照顺序计算$h_1,h_2,\cdots,h_T$即可。
当然我们时常需要在某一些时间刻输出一些预测值，比如输出一个多分类，那么我们还会对某一个时间刻的隐藏层再单独进行一次变换：
$$\hat y_t = SoftMax(W_o h_t + b_o)$$
接下来我们先导入以下这次需要使用的数据集————华尔街日报词性标注数据集，代码在loader.py中

解释一下这里的词性标注的规范：这里主要会出现的几种词性有NP（名词短语），VP（动词短语），PP（介词短语），ADVP（副词短语），ADJP（形容词短语），O（句子结束符号），PAD（占位符"<pad\>"对应的标注）.
除PAD和O外，剩下的标注都以连续几个单词构成的组块出现，比如(The big car --> B-NP, I-NP, I-NP)其中B前缀表示这个位置是这个名词短语中的第一个位置，I前缀表示这个位置是在该组块的中间或末尾。

In [1]:
from loader import get_dataloader, print_format

batch_size = 64
length_input = 30

train_loader = get_dataloader(mode="train", batch_size=batch_size, sentence_length=length_input)
dev_loader = get_dataloader(mode="test", batch_size=batch_size, sentence_length=length_input)

# Prints one line to visualize our dataset
text, tags = next(iter(dev_loader))
print_format(text[:,0], tags[:,0])

Loading dataset...
Counted 19124 words in dataset 
 Counted 23 output classes
 An /executive/model/would/significantly/boost/Jaguar/ 's /yearly/output/ of /50,000/cars/.
B-NP ---I-NP-- -I-NP -B-VP -----I-VP---- -I-VP -B-NP- B-NP -I-NP- -I-NP- B-PP -B-NP- I-NP O


## 基础循环神经网络RNN
接下来我们使用最为基础的循环神经网络来尝试解决这个问题，我们的网络的每一个时间刻的公式如下：
$$
\begin{align}
x_t &= \text{WordEmbedding}(w_t) \\
h_t &= \tanh(W_{ih}x_x + W_{hh}h_{t-1} + b_h) \\
\hat y_t &= \text{SoftMax}(W_o h_t + b_o) \\
\end{align}
$$
为了加速计算我们需要使用Mini-batch，出于简单性考虑，把所有长于30的文本都砍到30，把所有短于30的文本都使用"<pad\>"字符补充到30。（这些代码都体现在loader.py中）

In [2]:
# Define simple recurrent neural network module
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    """A ordinary recurrent neural network model

    :param vocab_size: The number of words in vocab
    :param output_size: The number of output classes
    :param hidden_size: The embedding dim and hidden units in rnn
    """
    def __init__(self, vocab_size, output_size, hidden_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn_linear = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, input):
        """Inputs in, Ouputs out, with all-zeros initial hidden units

        :param input: Inputs indexes with shape [sentence_length, batch_size]
        """
        sentence_length, batch_size = input.shape
        hiddens = torch.zeros((sentence_length, batch_size, self.hidden_size))
        embedded = self.embedding(input)
        
        # Iterates on every words
        for i in range(sentence_length):
            combine = torch.cat((embedded[i], hiddens[i-1]), dim=1)
            hiddens[i] = torch.tanh(self.rnn_linear(combine))
        outputs = self.out(hiddens)
        return self.softmax(outputs)

In [6]:
# Training
from train import train, evaluate
from loader import vocab_text, vocab_tag
rnn = RNN(len(vocab_text), len(vocab_tag), 128)
train(rnn, train_loader, dev_loader, 
    num_epoches=15, 
    log_interval=100, 
    learning_rate=1e-3)

Training starts...
Epoch: 1	Step: 99	Loss= 1.112	Acc: 0.815	0m:7s (- 2m:16s)
Epoch: 2	Step: 99	Loss= 0.488	Acc: 0.865	0m:16s (- 2m:0s)
Epoch: 3	Step: 99	Loss= 0.372	Acc: 0.890	0m:24s (- 1m:49s)
Epoch: 4	Step: 99	Loss= 0.302	Acc: 0.905	0m:33s (- 1m:39s)
Epoch: 5	Step: 99	Loss= 0.256	Acc: 0.915	0m:42s (- 1m:29s)
Epoch: 6	Step: 99	Loss= 0.220	Acc: 0.921	0m:50s (- 1m:20s)
Epoch: 7	Step: 99	Loss= 0.193	Acc: 0.928	0m:58s (- 1m:11s)
Epoch: 8	Step: 99	Loss= 0.169	Acc: 0.930	1m:7s (- 1m:2s)
Epoch: 9	Step: 99	Loss= 0.153	Acc: 0.932	1m:15s (- 0m:53s)
Epoch: 10	Step: 99	Loss= 0.135	Acc: 0.934	1m:24s (- 0m:44s)
Epoch: 11	Step: 99	Loss= 0.122	Acc: 0.934	1m:32s (- 0m:36s)
Epoch: 12	Step: 99	Loss= 0.108	Acc: 0.935	1m:40s (- 0m:27s)
Epoch: 13	Step: 99	Loss= 0.099	Acc: 0.935	1m:48s (- 0m:18s)
Epoch: 14	Step: 99	Loss= 0.088	Acc: 0.937	1m:56s (- 0m:10s)
Epoch: 15	Step: 99	Loss= 0.079	Acc: 0.937	2m:4s (- 0m:1s)
Training finishes!


## 长短期记忆模型LSTM
LSTM和RNN在基本结构上是一致的，但是在每一个单元的内容上有了比较大的变化，具体的方程为：
$$
\begin{align}
x_t &= \text{WordEmbedding}(w_t) \\
\tilde c ^{⟨t⟩}&=\tanh(W_c[a^{⟨t-1⟩},x^{⟨t⟩}]+b_c)\\
\Gamma_u &= \sigma(W_u[a^{⟨t-1⟩}, x^{⟨t⟩}]+b_u)\\
\Gamma_f &= \sigma(W_f[a^{⟨t-1⟩}, x^{⟨t⟩}]+b_f)\\
\Gamma_o&=\sigma(W_o[a^{⟨t-1⟩},x^{⟨t⟩}]+b_o)\\
c^{⟨t⟩} &= \Gamma_u \odot\tilde c^{⟨t⟩}+\Gamma_f\odot c^{⟨t-1⟩}\\
a^{⟨t⟩} &= \Gamma_o \odot \tanh(c^{⟨t⟩})\\
\hat y_t &= \text{SoftMax}(W_o a^{⟨t⟩} + b_o) \\
\end{align}
$$

接下来开始实现它（当然你可以直接使用nn.LSTM结束战斗）,之后我们使用同样的配置进行训练，可以看到相对于原来的RNN基本没有提升(?_?)，我认为可能的原因是词性标注任务并不是很需要长期记忆的帮助。

并且进一步实验发现，Pytorch的实现nn.LSTM比我们的实现快了三倍，这到底差在哪里呢？经过一番考虑，发现把计算embedding和最后的计算output的部分都移出循环，使用向量化的方法计算（原先是在每个循环中单独计算第t时间刻的embedded和outputs, 现在的版本是全部改为向量化操作了）就可以达到和官方实现差不多的速度，可见实现循环网络的时候一定要尽可能减少在循环体中的计算而多多利用向量化计算，速度上的变化是极为可观的。

In [4]:
# Define Long Short Term Memory Module
class LSTM(nn.Module):
    def __init__(self, vocab_size, output_size, hidden_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.forget_gate = nn.Linear(2 * hidden_size, hidden_size)
        self.input_gate = nn.Linear(2 * hidden_size, hidden_size)
        self.output_gate = nn.Linear(2 * hidden_size, hidden_size)
        self.cell_update = nn.Linear(2 * hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input):
        sentence_length, batch_size = input.shape
        hiddens = torch.zeros((sentence_length, batch_size, self.hidden_size))
        cell_state = torch.zeros((batch_size, self.hidden_size))
        embedded = self.embedding(input)

        for i in range(sentence_length):
            combine = torch.cat((embedded[i], hiddens[i-1]), dim=1)
            fg = torch.sigmoid(self.forget_gate(combine))
            ig = torch.sigmoid(self.input_gate(combine))
            og = torch.sigmoid(self.output_gate(combine))
            updated_cell = torch.sigmoid(self.cell_update(combine))

            cell_state = fg * cell_state + ig * updated_cell
            hiddens[i] = og * torch.sigmoid(cell_state)

        outputs = self.out(hiddens)
        outputs = self.softmax(outputs)

        return outputs

In [7]:
# Training
from train import train, evaluate
from loader import vocab_text, vocab_tag

lstm = LSTM(len(vocab_text), len(vocab_tag), 128)
train(lstm, train_loader, dev_loader, 
    num_epoches=14, 
    log_interval=100, 
    learning_rate=1e-3)

Training starts...
Epoch: 1	Step: 99	Loss= 1.443	Acc: 0.748	0m:10s (- 2m:56s)
Epoch: 2	Step: 99	Loss= 0.617	Acc: 0.859	0m:22s (- 2m:33s)
Epoch: 3	Step: 99	Loss= 0.426	Acc: 0.885	0m:34s (- 2m:18s)
Epoch: 4	Step: 99	Loss= 0.347	Acc: 0.899	0m:46s (- 2m:5s)
Epoch: 5	Step: 99	Loss= 0.297	Acc: 0.908	0m:58s (- 1m:52s)
Epoch: 6	Step: 99	Loss= 0.260	Acc: 0.917	1m:10s (- 1m:39s)
Epoch: 7	Step: 99	Loss= 0.234	Acc: 0.922	1m:22s (- 1m:27s)
Epoch: 8	Step: 99	Loss= 0.213	Acc: 0.925	1m:34s (- 1m:15s)
Epoch: 9	Step: 99	Loss= 0.192	Acc: 0.927	1m:46s (- 1m:2s)
Epoch: 10	Step: 99	Loss= 0.177	Acc: 0.931	1m:58s (- 0m:50s)
Epoch: 11	Step: 99	Loss= 0.161	Acc: 0.933	2m:12s (- 0m:39s)
Epoch: 12	Step: 99	Loss= 0.152	Acc: 0.934	2m:24s (- 0m:27s)
Epoch: 13	Step: 99	Loss= 0.140	Acc: 0.934	2m:35s (- 0m:14s)
Epoch: 14	Step: 99	Loss= 0.131	Acc: 0.934	2m:48s (- 0m:2s)
Training finishes!


## 门控循环单元模型GRU
GRU可以说是LSTM的简化版本，LSTM有三个门控，而GRU只有两个，当然还有只有一个门控单元的如MGU，这里不过多介绍。GRU把LSTM的最为精髓的部分保留了：
$$
\begin{align}
\Gamma_r &= \sigma(W_r[c^{⟨t-1⟩}, x^{⟨t⟩}] + b_r)\\
\tilde c^{⟨t⟩} &= tanh(W_c[\Gamma_r\odot c^{⟨t-1⟩}, x^{⟨t⟩}] + b_c)\\

\Gamma_u &= \sigma(W_u[c^{⟨t-1⟩}, x^{⟨t⟩}] + b_u)\\
c^{⟨t⟩} &= \Gamma_u \odot \tilde c^{⟨t⟩} + (1 - \Gamma_u) \odot c^{⟨t-1⟩}
\end{align}
$$
经过完全一样的训练，我们发现GRU还是非常香的，不仅收敛速度快了一些，而且最终的验证集准确率也是还可以。

Pytorch中一个值得注意的细节：
```
RuntimeError: one of the variables needed for gradient computation has been modified 
by an inplace operation: [torch.FloatTensor ], which is output 0 of UnsqueezeBackward0, 
is at version 3; expected version 2 instead. Hint: enable anomaly detection to find 
the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).
```
这个错误常常出现于`x[i] = self.some_module(x[i])`中，而`x = self.some_module(x)`则不会报这样的错误。
这个错误告诉我们不要轻易在forward函数中使用tensor的部分元素赋值。因为Pytorch依赖追踪forward过程中的中间变量的引用来实现自动微分，但是tensor的部分元素赋值是按值传递，如果原先这个位置上的值已经有梯度，那么赋值也将会覆盖掉这个梯度，使得反向传播的链条就此断开。但是`x = self.some_module(x)`不会出问题的原因是x的赋值只是获得的新的值的引用，而原来的x还好好的。

如果确实需要覆盖数组元素，那么就要保证覆盖前的元素没有参与运算，且覆盖后的元素也不会再被覆盖。

In [8]:
# Define Gate Recurrent Unit Module
class GRU(nn.Module):
    def __init__(self, vocab_size, output_size, hidden_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.update_gate = nn.Linear(hidden_size * 2, hidden_size)
        self.reset_gate = nn.Linear(hidden_size * 2, hidden_size)
        self.cell_update = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input):
        sentence_length, batch_size = input.shape
        state = torch.zeros((batch_size, self.hidden_size))
        hiddens = torch.zeros((sentence_length, batch_size, self.hidden_size))
        embedded = self.embedding(input)

        for i in range(sentence_length):
            combine = torch.cat((embedded[i], state), dim=1)
            ug = torch.sigmoid(self.update_gate(combine))
            rg = torch.sigmoid(self.reset_gate(combine))
            state_gated = rg * state
            reseted_combine = torch.cat((embedded[i], state_gated), dim=1)
            state_updated = torch.tanh(self.cell_update(reseted_combine))
            state = ug * state_updated + (1 - ug) * state

            # Vectorizes and avoid of inplace operation error meanwhile
            hiddens[i] = state
        outputs = self.out(hiddens)
        return self.softmax(outputs)

In [40]:
# Training
from train import train, evaluate
from loader import vocab_text, vocab_tag

gru = GRU(len(vocab_text), len(vocab_tag), 128)
train(gru, train_loader, dev_loader, 
    num_epoches=14, 
    log_interval=100, 
    learning_rate=1e-3)

Training starts...
Epoch: 1	Step: 99	Loss= 1.629	Acc: 0.722	0m:5s (- 1m:31s)
Epoch: 2	Step: 99	Loss= 0.682	Acc: 0.828	0m:12s (- 1m:27s)
Epoch: 3	Step: 99	Loss= 0.490	Acc: 0.863	0m:19s (- 1m:17s)
Epoch: 4	Step: 99	Loss= 0.396	Acc: 0.884	0m:25s (- 1m:9s)
Epoch: 5	Step: 99	Loss= 0.337	Acc: 0.895	0m:32s (- 1m:2s)
Epoch: 6	Step: 99	Loss= 0.298	Acc: 0.905	0m:38s (- 0m:55s)
Epoch: 7	Step: 99	Loss= 0.263	Acc: 0.911	0m:45s (- 0m:48s)
Epoch: 8	Step: 99	Loss= 0.239	Acc: 0.916	0m:52s (- 0m:41s)
Epoch: 9	Step: 99	Loss= 0.216	Acc: 0.920	0m:58s (- 0m:34s)
Epoch: 10	Step: 99	Loss= 0.197	Acc: 0.923	1m:5s (- 0m:28s)
Epoch: 11	Step: 99	Loss= 0.180	Acc: 0.926	1m:11s (- 0m:21s)
Epoch: 12	Step: 99	Loss= 0.165	Acc: 0.928	1m:18s (- 0m:14s)
Epoch: 13	Step: 99	Loss= 0.154	Acc: 0.929	1m:24s (- 0m:8s)
Epoch: 14	Step: 99	Loss= 0.139	Acc: 0.930	1m:31s (- 0m:1s)
Training finishes!


## 双向LSTM（Bi-Directional LSTM）
双向SLTM就是两个LSTM，一个从$t=1$计算到$t=T$，一个是从$t=T$计算到$t=1$。而用于输出的全连接神经网络则在任何一个时刻都接受两个方向的LSTM单元在同一个时间刻输出的隐藏层数值进行处理。由于不再涉及新的公式，这里直接使用nn.LSTM解决了。事实证明，即使是双向LSTM也不能很好的提高这个问题上的表现。

In [8]:
take = type('', (nn.Module,), dict(forward = lambda self, x: x[0]))()
take([1,2])

1

In [ ]:
# Define model and training.
# Ugly but short code
from train import train, evaluate
from loader import vocab_text, vocab_tag

hidden_size = 128

bi_lstm = nn.Sequential(
    nn.Embedding(len(vocab_text), hidden_size),
    nn.LSTM(hidden_size, hidden_size, bidirectional=True),
    type('TakeFirst', (nn.Module,), dict(forward = lambda self, x: x[0]))(), # 什么叫做动态语言啊（后仰）
    nn.Linear(hidden_size * 2, len(vocab_tag)),
    nn.LogSoftmax(dim=2)
)
bi_lstm.__setattr__('output_size', len(vocab_tag))

train(bi_lstm, train_loader, dev_loader, 
    num_epoches=14, 
    log_interval=100, 
    learning_rate=1e-3)

## 总结
对于这个词性标注的问题，看来更加复杂的网络除了增加训练时间，也都没办法对训练造成什么影响。让我们输出几个验证集中的错误例子看看到底是怎样的错误是如此的顽固不化。

In [49]:
# Yields out mismatched text
# Returns an generator of wrongly tagged data
def yield_mistakes(model, data_loader):
    for inputs, targets in data_loader:
        outputs = model(inputs)
        preds = outputs.argmax(dim=2)
        batch_size = preds.size(1)
        for i in range(batch_size):
            input = inputs[:, i]
            target = targets[:, i]
            pred = preds[:, i]
            if target.numel() != torch.sum(pred == target):
                yield input, target, pred
wrong_instances = yield_mistakes(lstm, dev_loader)

In [52]:
# Run this repeatly to visualize some wrongly tagged pieces
text, tags, pred = next(wrong_instances)
print_format(text, tags, pred)

 A  /pact/with/ GM /may /emerge/ in / as /little/ as /two /weeks/,/according/ to /sources/close / to /the /talks/.
B-NP I-NP B-PP B-NP B-VP -I-VP- B-PP B-NP -I-NP- I-NP I-NP -I-NP O ---B-PP-- B-PP --B-NP- B-ADJP B-PP B-NP -I-NP O
B-NP I-NP B-PP B-NP B-VP -I-VP- B-PP B-NP -I-NP- I-NP B-NP -I-NP O ---B-PP-- B-PP --B-NP- -I-NP- B-PP B-NP -I-NP O


经过对验证集和训练集中的错误数据进行观察以后，我们可以得出一个比较有信服力的解释————算法对于词语在文本中的地位并不理解。比如“before”可以表示副词“I did it before”或表示介词“before it happens”。 或者数词也可以作形容词成分“10 happy kids”，或者作名词成分“score increased 10%”，这都是词语在文本中的地位不同导致的。算法实际上没能很好地从文本中提取出不同组分之间的地位关系。
上面提到的只是语句理解方面的问题，而影响正确率的另一个问题是算法对标记规则依然并不理解，比如算法经常会把I-XP置于一个短语组块的开头，这显然是不对的，任何短语组块的标记都应该形如“B-XP”或“B-XP, I-XP, ...”。解决这个问题我们如果想等待网络自己搞明白这件事那就太花时间了，一种可行的方法是在网络的末尾加入一个CRF层，它的主要思想是Bi-LSTM只能观察不同输入之间的关系而得到一个输出标签，但是它无法观察到其他预测的标签，自然难以协调不同标签之间的前后关系。CRF层就可以监督网络输出符合规则的标签。当然由于Pytorch没有实现CRF层，这里我们就不进行实验了。